In [1]:
# imports
import pyodbc
import pandas as pd
from datetime import date, datetime

In [2]:
# digitenis database connection - linux connection
dsn = 'sqlserverdatasource'
user = 'sa'
password = 'digiTenis1@'
database = 'digitenis'

conn_string = 'DSN=%s;UID=%s;PWD=%s;DATABASE=%s;' % (dsn, user, password, database)
conn = pyodbc.connect(conn_string)

In [ ]:
# digitenis database connection - windows connection (localhost)
conn_dados = (
    "Driver={SQL Server};"
    "Server=cmd->hostname;"
    "Database=digitenis"
)

conn = pyodbc.connect(conn_dados)

In [ ]:
# dataframe tables
fornecedor = pd.read_sql('select * from fornecedor', conn)
telefone = pd.read_sql('select * from telefone', conn)
funcionario = pd.read_sql('select * from funcionario', conn)
produto = pd.read_sql('select * from produto', conn)
venda = pd.read_sql('select * from venda', conn)
item_venda = pd.read_sql('select * from item_venda', conn)
estoque = pd.read_sql('select * from estoque', conn)

## Functional Requirements

In [11]:
# cadastrar novos fornecedores
fornecedor.loc[len(fornecedor)] = [21, '6478912300015', 'Olympikus', 'contato@olympikus.com.br']

In [13]:
# cadastrar novos produtos
produto.loc[len(produto)] = [21, 'Anyway 2', 39, 'Corrida', 80, 119.90, 21]

In [14]:
# cadastrar novos funcionários
funcionario.loc[len(funcionario)] = [19, 'Gisele', 'Oliveira', '45695135726', 'Vendedor']

In [28]:
# efetivar vendas
venda.loc[len(venda)] = [29, 359.70, date.today(), datetime.now().time(), 19]

item_venda.loc[len(item_venda)] = [21, 29, 3, 35.97]


In [ ]:
# visualizar informações de contato dos fornecedores
fornecedor[['nome', 'email']]

In [43]:
# atualizar valor de venda dos produtos
produto.loc[produto['id_produto'] == 21, 'valor_venda'] = 323.73

In [ ]:
#visualizar estoque por produto
p_e = produto.merge(estoque, how="inner", on="id_produto")

p_e[["id_produto", "nome", "tamanho", "quantidade"]].loc[p_e["id_produto"] == 20]

In [ ]:
#visualizar vendas do dia
v_iv = venda.merge(item_venda, how="inner", on="id_venda")

v_iv_p = v_iv.merge(produto, how="inner", on="id_produto")

v_iv_p[["data", "nome", "quantidade", "desconto", "valor_total"]].loc[v_iv_p["data"] == '2023-03-21']

In [ ]:
#visualizar dias com maiores vendas
venda[["data", "valor_total"]].groupby(by="data").sum("valor_total").sort_values(by="valor_total", ascending=False)

In [ ]:
#visualizar funcionarios com maiores vendas
v_f = venda.merge(funcionario, how="inner", on="id_funcionario")

v_f[["primeiro_nome", "valor_total"]].groupby(by="primeiro_nome").sum("valor_total").sort_values(by="valor_total", ascending=False)

In [ ]:
#visualizar lista dos itens comprados por venda
iv_p = item_venda.merge(produto, how="inner", on="id_produto")
iv_p_v = iv_p.merge(venda, how="inner", on="id_venda")

iv_p_v[["id_venda", "nome", "tamanho", "categoria", "data", "quantidade"]].sort_values("id_venda")

In [ ]:
#visualizar lista dos itens mais vendidos por funcionario
v_iv = venda.merge(item_venda, how='inner', on='id_venda')
v_iv_p = v_iv.merge(produto, how='inner', on='id_produto')
v_iv_p_f = v_iv_p.merge(funcionario, how='inner', on='id_funcionario')

v_iv_p_f[["primeiro_nome", "nome", "quantidade"]].groupby(by=["primeiro_nome", "nome"]).sum("quantidade").sort_values("quantidade", ascending=False)

In [ ]:
#Visualizar média de saída de cada produto por dia
iv_v = item_venda.merge(venda, how='inner', on='id_venda')
iv_v_p = iv_v.merge(produto, how='inner', on='id_produto')
iv_v_p[["nome", "quantidade"]].groupby(by="nome").mean("quantidade")

In [ ]:
#Calcular a quantidade de dias em que o estoque deve romper, considerando a média de saída diária dos produtos
join1 = item_venda.merge(venda, how='inner', on='id_venda')
join2 = join1.merge(produto, how='inner', on='id_produto')
dados1 = join2[["id_produto", "quantidade"]].groupby(by="id_produto").mean("quantidade")
join3 = dados1.merge(estoque, how='inner', on='id_produto')
join3['previsao_dias_rompimento'] = join3['quantidade_y'] / join3['quantidade_x']

In [ ]:
#Atualizar dados cadastrais dos fornecedores
fornecedor.loc[fornecedor['id_fornecedor'] == 20, 'email'] = 'contato2@fila.com.br'

In [ ]:
#Atualizar dados cadastrais dos funcionários
funcionario.loc[funcionario['id_funcionario'] == 12, 'cargo'] = 'Gerente'

In [ ]:
#Visualizar média dos valores de vendas
venda["valor_total"].mean()

In [ ]:
#Visualizar média de itens por venda
join1 = venda.merge(item_venda, how='inner', on='id_venda')
join1['total_vendas'] = join1['id_venda'].count()
join1['total_itens_vendidos'] = join1['quantidade'].sum()
join1['media_itens_por_venda'] = join1['total_itens_vendidos'] / join1['total_vendas']
join1[['total_vendas', 'total_itens_vendidos', 'media_itens_por_venda']]

In [ ]:
#Visualizar categoria de produto que mais é vendido
join1 = item_venda.merge(produto, how='inner', on='id_produto')
join1[['categoria', 'quantidade']].groupby(by='categoria').sum('quantidade').sort_values(by='quantidade', ascending=False)

In [ ]:
#Dar baixa no estoque
estoque.loc[estoque['id_produto'] == 3, 'quantidade'] -= 1
estoque

In [ ]:
#Dar entrada no estoque
estoque.loc[estoque['id_produto'] == 3, 'quantidade'] += 1
estoque